# Image classification 
Task: Predict disease type of bean plants.
Dataset: Beans from TFDS

## 1. Loading dataset

In [56]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf

In [55]:
(ds_train, ds_test), ds_info = tfds.load('beans', split = ['train', 'test[:20%]'], shuffle_files=True, as_supervised=True, with_info=True)

## 2. Preprocessing dataset
Includes: 
- resizing
- rotation
- batching

In [57]:
def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., label

def resize_rescale(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.reshape(image, [-1])
    return image, label


ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

Epoch 1/6
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 734ms/step - loss: 272.7516 - sparse_categorical_accuracy: 0.2411 - val_loss: 457.0703 - val_sparse_categorical_accuracy: 0.2692
Epoch 2/6
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 706ms/step - loss: 276.3286 - sparse_categorical_accuracy: 0.3638 - val_loss: 86.0862 - val_sparse_categorical_accuracy: 0.3846
Epoch 3/6
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 705ms/step - loss: 104.3876 - sparse_categorical_accuracy: 0.4303 - val_loss: 41.0458 - val_sparse_categorical_accuracy: 0.5000
Epoch 4/6
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 706ms/step - loss: 50.8693 - sparse_categorical_accuracy: 0.5526 - val_loss: 59.9973 - val_sparse_categorical_accuracy: 0.6154
Epoch 5/6
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 707ms/step - loss: 33.2931 - sparse_categorical_accuracy: 0.5872 - val_loss: 39.5069 - val_sparse_categorical_accuracy: 0.7308
Epoch 6/6
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 711ms/step - loss: 24.2419 - sparse_categorical_accuracy: 0.6345 - val_loss: 21.1397 - val_sparse_categorical_accuracy: 0.5769
